In [ ]:
!git clone https://github.com/eliogalindo92/Actividades.git
import cv2
import random
import numpy as np
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
clear_output()

# Cargo la imagen de disco
gray_image = cv2.imread('/content/Actividades/Semana 3/Recursos/ariana_grande.jpg', cv2.IMREAD_GRAYSCALE)
damaged_image = cv2.imread('/content/Actividades/Semana 3/Recursos/damaged_image.jpg')

In [1]:
# Inciso a

# Función para ecualizar el histograma.
equalized_image = cv2.equalizeHist(gray_image)
equalization_view = np.hstack((gray_image, equalized_image))
cv2_imshow(equalization_view)

NameError: name 'cv2' is not defined

In [ ]:
# Inciso b

# Función para binarizar la imagen aplicando THRESH_OTSU.
threshold, thresholded_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_OTSU)
OTSU_threshold_view = np.hstack((gray_image, thresholded_image))
cv2_imshow(OTSU_threshold_view)

In [ ]:
# Inciso c

# Función para eliminar agujeros en la imagen.
threshold = cv2.inRange(damaged_image, np.array([240, 240, 240]), np.array([255, 255, 255]))
kernel = np.ones((3, 3), np.uint8)
hi_mask = cv2.dilate(threshold, kernel, iterations=10)
fixed_image = cv2.inpaint(damaged_image, hi_mask, 5, flags=cv2.INPAINT_TELEA)
fixed_image_view = np.hstack((damaged_image, fixed_image))
cv2_imshow(fixed_image_view)

In [ ]:
# Inciso d

# Función para segmentar una imagen con k = 8.
image_copy = np.copy(fixed_image)
pixel_values = image_copy.reshape((-1, 3))
pixel_values = np.float32(pixel_values)
k = 8
stop_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
number_of_attempts = 10
centroid_initialization_strategy = cv2.KMEANS_RANDOM_CENTERS
_, labels, centers = cv2.kmeans(pixel_values, k, None, stop_criteria, number_of_attempts, centroid_initialization_strategy)
centers = np.uint8(centers)
segmented_data = centers[labels.flatten()]
segmented_image = segmented_data.reshape(image_copy.shape)
segmentation_view = np.hstack((fixed_image, segmented_image))
cv2_imshow(segmentation_view)